## Asynchronous Video Capture

* Code directly taken from Intel® Distribution of OpenVINO™ Toolkit

In [9]:
import cv2
import time
import threading

In [10]:
class VideoPlayer:

    def __init__(self, source, fps=None):

        self.__cap = cv2.VideoCapture(source)
        if not self.__cap.isOpened():
            raise RuntimeError(
                f"Cannot open {'camera' if isinstance(source, int) else ''} {source}"
            )
        # fps of input file
        self.__input_fps = self.__cap.get(cv2.CAP_PROP_FPS)
        if self.__input_fps <= 0:
            self.__input_fps = 60
        # target fps given by user
        self.__output_fps = fps if fps is not None else self.__input_fps
        self.__size = None
        
        # first frame
        _, self.__frame = self.__cap.read()
        self.__lock = threading.Lock()
        self.__thread = None
        self.__stop = False

    def start(self):
        self.__stop = False
        self.__thread = threading.Thread(target=self.__run, daemon=True)
        self.__thread.start()

    def stop(self):
        self.__stop = True
        if self.__thread is not None:
            self.__thread.join()
        self.__cap.release()

    def __run(self):
        prev_time = 0
        while not self.__stop:
            t1 = time.time()
            ret, frame = self.__cap.read()
            if not ret:
                break

            # fulfill target fps
            if 1 / self.__output_fps < time.time() - prev_time:
                prev_time = time.time()
                # replace by current frame
                with self.__lock:
                    self.__frame = frame

            t2 = time.time()
            # time to wait [s] to fulfill input fps
            wait_time = 1 / self.__input_fps - (t2 - t1)
            # wait until
            time.sleep(max(0, wait_time))

        self.__frame = None

    def next(self):
        with self.__lock:
            if self.__frame is None:
                return None
            # need to copy frame, because can be cached and reused if fps is low
            frame = self.__frame.copy()
        return frame

In [11]:
def asynchronous_video_capture(source, fps=60):
    
    player = None

    try:
        # Create a video player
        player = VideoPlayer(source, fps=fps)
        # Start capturing
        start_time = time.time()
        player.start()

        frame_number = 0
        while True:
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            stop_time = time.time()
            total_time = stop_time - start_time
            frame_number = frame_number + 1
            sync_fps = frame_number / total_time

            cv2.putText(frame, "Asynchronous Video Capture", (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, str(round(sync_fps, 2)) + " FPS", (5, 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
            
            cv2.namedWindow("Asynchronous Video Capture", cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
            cv2.imshow("Asynchronous Video Capture", frame)
            
            key = cv2.waitKey(1)
            # escape = 27
            if key == 27:
                break
            
    except KeyboardInterrupt:
        print("Interrupted")
    # Any different error
    except RuntimeError as e:
        print(e)
    finally:
        cv2.destroyAllWindows()
        if player is not None:
            # stop capturing
            player.stop()


In [12]:
asynchronous_video_capture(source=4)